In [1]:
using CUDA
using GLMakie
include("Operations.jl")

run (generic function with 1 method)

In [2]:
const nx = 41
const ny = 41
const nt = 10
const nit = 51
const c = 1
const dx = 2 / (nx - 1)
const dy = 2 / (ny - 1)

CUDA.allowscalar(false)

In [3]:
x = CuArray{Float64}([i for i in range(0.0, stop=2.0, step=dx)])
y = CuArray{Float64}([i for i in range(0.0, stop=2.0, step=dy)])

41-element CuArray{Float64, 1, CUDA.Mem.DeviceBuffer}:
 0.0
 0.05
 0.1
 0.15
 0.2
 0.25
 0.3
 0.35
 0.4
 0.45
 ⋮
 1.6
 1.65
 1.7
 1.75
 1.8
 1.85
 1.9
 1.95
 2.0

In [4]:
const rho = 1
const nu = 0.1
const F = 1
const dt = 0.01

0.01

In [5]:
u = CuArray{Float64}(undef, nx, ny)
u = CUDA.zeros(nx, ny)
v = CuArray{Float64}(undef, nx, ny)
v = CUDA.zeros(nx, ny)
p = CuArray{Float64}(undef, nx, ny)
p = CUDA.zeros(nx, ny)
b = CuArray{Float64}(undef, nx, ny)
b = CUDA.zeros(nx, ny)
un = CuArray{Float64}(undef, nx, ny)
un = CUDA.zeros(nx, ny)
vn = CuArray{Float64}(undef, nx, ny)
vn = CUDA.zeros(nx, ny)
pn = CuArray{Float64}(undef, nx, ny)
pn = CUDA.zeros(nx, ny)

41×41 CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 ⋮                        ⋮              ⋱       ⋮                        ⋮
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  

In [7]:
f = Figure(resolution = (800, 800))
Axis(f[1,1], backgroundcolor = "black")

#GLMakie.record(f, "Channel Flow GPU.gif", 1:499) do i
for i in range(1, stop=500)
    un[:,:] = u[:,:]
    vn[:,:] = v[:,:]

    u[2:nx-1, 2:nx-1] = (un[2:nx-1, 2:nx-1] -
                         un[2:nx-1, 2:nx-1] .* dt / dx .* 
                        (un[2:nx-1, 2:nx-1] - un[2:nx-1, 1:nx-2]) -
                         vn[2:nx-1, 2:nx-1] .* dt / dy .* 
                        (un[2:nx-1, 2:nx-1] - un[1:nx-2, 2:nx-1]) -
                         dt / (2 .* rho .* dx) .* 
                        (p[2:nx-1, 3:nx] - p[2:nx-1, 1:nx-2]) .+
                         nu .* (dt / dx^2 .* 
                        (un[2:nx-1, 3:nx] - 2 .* un[2:nx-1, 2:nx-1] .+ un[2:nx-1, 1:nx-2]) .+
                         dt / dy^2 .* 
                        (un[3:nx, 2:nx-1] - 2 .* un[2:nx-1, 2:nx-1] .+ un[1:nx-2, 2:nx-1])) .+ 
                         F .* dt)
    
        v[2:nx-1, 2:nx-1] = (vn[2:nx-1, 2:nx-1] -
                         un[2:nx-1, 2:nx-1] .* dt / dx .* 
                        (vn[2:nx-1, 2:nx-1] - vn[2:nx-1, 1:nx-2]) -
                         vn[2:nx-1, 2:nx-1] .* dt / dy .* 
                        (vn[2:nx-1, 2:nx-1] - vn[1:nx-2, 2:nx-1]) -
                         dt / (2 .* rho .* dy) .* 
                        (p[3:nx, 2:nx-1] - p[1:nx-2, 2:nx-1]) .+
                         nu .* (dt / dx^2 .*
                        (vn[2:nx-1, 3:nx] - 2 .* vn[2:nx-1, 2:nx-1] .+ vn[2:nx-1, 1:nx-2]) .+
                         dt / dy^2 .* 
                        (vn[3:nx, 2:nx-1] - 2 .* vn[2:nx-1, 2:nx-1] .+ vn[1:nx-2, 2:nx-1])))
    
        # Periodic BC u @ x = 2     
        u[2:nx-1, nx-1] = (un[2:nx-1, nx-1] - un[2:nx-1, nx-1] .* dt / dx .* 
                      (un[2:nx-1, nx-1] - un[2:nx-1, nx-2]) -
                       vn[2:nx-1, nx-1] .* dt / dy .* 
                      (un[2:nx-1, nx-1] - un[1:nx-2, nx-1]) -
                       dt / (2 .* rho .* dx) .*
                      (p[2:nx-1, 1] - p[2:nx-1, nx-2]) .+ 
                       nu .* (dt / dx^2 .* 
                      (un[2:nx-1, 1] - 2 .* un[2:nx-1, nx-1] .+ un[2:nx-1, nx-2]) .+
                       dt / dy^2 .* 
                      (un[3:nx, nx-1] - 2 .* un[2:nx-1, nx-1] .+ un[1:nx-2, nx-1])) .+ F .* dt)
    
        # Periodic BC u @ x = 0
        u[2:nx-1, 1] = (un[2:nx-1, 1] - un[2:nx-1, 1] .* dt / dx .*
                     (un[2:nx-1, 1] - un[2:nx-1, nx-1]) -
                      vn[2:nx-1, 1] .* dt / dy .* 
                     (un[2:nx-1, 1] - un[1:nx-2, 1]) - 
                      dt / (2 .* rho .* dx) .* 
                     (p[2:nx-1, 2] - p[2:nx-1, nx-1]) .+ 
                      nu .* (dt / dx^2 .* 
                     (un[2:nx-1, 2] - 2 .* un[2:nx-1, 1] .+ un[2:nx-1, nx-1]) .+
                      dt / dy^2 .*
                     (un[3:nx, 1] - 2 .* un[2:nx-1, 1] .+ un[1:nx-2, 1])) .+ F .* dt)
    
        # Periodic BC v @ x = 2
        v[2:nx-1, nx-1] = (vn[2:nx-1, nx-1] - un[2:nx-1, nx-1] .* dt / dx .*
                      (vn[2:nx-1, nx-1] - vn[2:nx-1, nx-2]) - 
                       vn[2:nx-1, nx-1] .* dt / dy .*
                      (vn[2:nx-1, nx-1] - vn[1:nx-2, nx-1]) -
                       dt / (2 .* rho .* dy) .* 
                      (p[3:nx, nx-1] - p[1:nx-2, nx-1]) .+
                       nu .* (dt / dx^2 .*
                      (vn[2:nx-1, 1] - 2 .* vn[2:nx-1, nx-1] .+ vn[2:nx-1, nx-2]) .+
                       dt / dy^2 .*
                      (vn[3:nx, nx-1] - 2 .* vn[2:nx-1, nx-1] .+ vn[1:nx-2, nx-1])))
    
        # Periodic BC v @ x = 0
        v[2:nx-1, 1] = (vn[2:nx-1, 1] - un[2:nx-1, 1] .* dt / dx .*
                     (vn[2:nx-1, 1] - vn[2:nx-1, nx-1]) -
                      vn[2:nx-1, 1] .* dt / dy .*
                     (vn[2:nx-1, 1] - vn[1:nx-2, 1]) -
                      dt / (2 .* rho .* dy) .* 
                     (p[3:nx, 1] - p[1:nx-2, 1]) .+
                      nu .* (dt / dx^2 .* 
                     (vn[2:nx-1, 2] - 2 .* vn[2:nx-1, 1] .+ vn[2:nx-1, nx-1]) .+
                      dt / dy^2 .* 
                     (vn[3:nx, 1] - 2 .* vn[2:nx-1, 1] .+ vn[1:nx-2, 1])))
    
    
        # Wall BC: u,v = 0 @ y = 0,2
        u[1, :] .= 0.0
        u[nx-1, :] .= 0.0
        v[1, :] .= 0.0
        v[nx-1, :] .= 0.0

        strength = vec(sqrt.(u[2:nx-2, 2:nx-2] .^ 2 .+ v[2:nx-2, 2:nx-2] .^ 2))
        arrows!(x[2:nx-2], y[2:nx-2], v[2:nx-2, 2:nx-2], u[2:nx-2, 2:nx-2], lengthscale = 0.01, arrowcolor = strength, linecolor = strength)
end

GPUCompiler.KernelError: GPU compilation of kernel broadcast_kernel(CUDA.CuKernelContext, CuDeviceVector{Pair{Point{2, Float32}, Point{2, Float32}}, 1}, Base.Broadcast.Broadcasted{Nothing, Tuple{Base.OneTo{Int64}}, Makie.var"#324#343"{Bool, Symbol, Float64}, Tuple{Base.Broadcast.Extruded{CuDeviceVector{Point{2, Float32}, 1}, Tuple{Bool}, Tuple{Int64}}, Base.Broadcast.Extruded{CuDeviceVector{Vec{2, Float32}, 1}, Tuple{Bool}, Tuple{Int64}}}}, Int64) failed
KernelError: passing and using non-bitstype argument

Argument 4 to your kernel function is of type Base.Broadcast.Broadcasted{Nothing, Tuple{Base.OneTo{Int64}}, Makie.var"#324#343"{Bool, Symbol, Float64}, Tuple{Base.Broadcast.Extruded{CuDeviceVector{Point{2, Float32}, 1}, Tuple{Bool}, Tuple{Int64}}, Base.Broadcast.Extruded{CuDeviceVector{Vec{2, Float32}, 1}, Tuple{Bool}, Tuple{Int64}}}}, which is not isbits:
  .f is of type Makie.var"#324#343"{Bool, Symbol, Float64} which is not isbits.
    .align is of type Symbol which is not isbits